# Cleaning and Pre-Processing Airbnb Data
<p> Now that we have a good understanding of what our data looks like, the Airbnb datasets provided need to be cleaned and edited for optimal model usage. This includes performing initial feature selection, imputing missing data, examining collinearity, performing variable transformations, and further pre-processing.</p>

### Import Libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline

In [4]:
# Global variables
BNB_BLUE = '#007A87'
BNB_RED = '#FF5A5F'
BNB_DARK_GRAY = '#565A5C'
BNB_LIGHT_GRAY = '#CED1CC'

### Initial Feature Selection
<p>As a baseline, we can start by removing features that we intuitively sense will not impact a listing's price. This includes 18 features:</p>
* `scrape_id`: Irrelevant to property data.
* `last_scraped`: All within first three days of January, irrelevant to actual property data.
* `picture_url`: Irrelevant to property data. 
* `host_id`: Irrelevant to property data.
* `host_name`: Irrelevant to property data; no textual importance.
* `host_since`: Irrelevant to property data; no textual importance.
* `host_picture_url`: Irrelevant to property data.
* `street`: Generic names; location data captured with lower unique count in other geographical features.
* `neighbourhood`: The `neighbourhood_cleansed` feature presents the same data in a better format.
* `state`: All listings are in the state of NY - this is useless.
* `market`: All listings are in the NY market - this is useless.
* `country`: All listings are in the USA - this is useless.
* `weekly_price`: Function of daily price - should not be a predictor.
* `monthly_price`: Function of daily price - should not be a predictor.
* `calendar_updated`: Irrelevant to property data.
* `calendar_last_scraped`: All within first three days of January, irrelevant to actual property data.
* `first_review`: Irrelevant to property data, high unique count.
* `last_review`: Irrelevant to property data, high unique count.

In [10]:
# Read in the data 
listings = pd.read_csv('listings_Jan.csv', delimiter=',')
calendar = pd.read_csv('calendar_Jan.csv', delimiter=',', usecols=range(4))

# Split into predictor and response
y = listings[['price']]

# Append price at the end of the listings table
del listings['price']
x = listings
listings = listings.join(y)

In [6]:
# Features to drop
bad_features = ['scrape_id', 'last_scraped', 'picture_url', 'host_picture_url', 
                'host_id', 'neighbourhood', 'state', 'market', 'country',
                'weekly_price', 'monthly_price', 'calendar_last_scraped',
                'host_name', 'host_since', 'street', 'calendar_updated',
                'first_review', 'last_review']

listings.drop(bad_features, axis=1, inplace=True)

# Store number of entries and features
entries = listings.shape[0]
features = listings.shape[1]

print 'Number of entries:',entries
print 'Number of features:',features
listings.head(n=3)

Number of entries: 27392
Number of features: 34


,id,name,neighbourhood_cleansed,city,zipcode,latitude,longitude,is_location_exact,property_type,room_type,...,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,host_listing_count,price
0,1069266,Stay like a real New Yorker!,Midtown East,New York,10022,40.756852,-73.964754,t,Apartment,Entire home/apt,...,62,86.0,9.0,7.0,9.0,9.0,10.0,9.0,1,$160.00
1,1846722,Apartment 20 Minutes Times Square,Hamilton Heights,New York,NaN,40.830599,-73.941014,f,Apartment,Entire home/apt,...,22,85.0,8.0,8.0,9.0,8.0,7.0,8.0,2,$105.00
2,2061725,Option of 2 Beds w Private Bathroom,Bushwick,Brooklyn,11221,40.692189,-73.924120,t,Apartment,Private room,...,35,98.0,10.0,10.0,10.0,10.0,9.0,10.0,4,$58.00


### Cleaning and Imputation
<p>We begin by making sure that all quantitative predictors and response variables are float. This will allow us to better deal with categorical data, and NaN entries in the float data. Looking through `zipcode` and `city` we realize that there is a lot of erroneous and incomplete data in these features, . Both of these

Before we can on-hot encode, we need to deal with our missing values. For categorical values, we will fill in using the mode because there is no ordering, and for quantitative variables we will use the median (to prevent outliers from skewing the imputation).</p>


In [7]:
# Convert to floats
listings['price'] = listings['price'].apply(lambda s: float(s[1:].replace(',','')))
listings['extra_people'] = listings['extra_people'].apply(lambda s: float(s[1:].replace(',','')))

In [8]:
# List of columns to be converted to floating point
to_float = ['id', 'latitude', 'longitude', 'accommodates',
            'bathrooms', 'bedrooms', 'beds', 'square_feet',
            'guests_included', 'extra_people', 'minimum_nights', 
            'maximum_nights', 'availability_30', 'availability_60', 
            'availability_90', 'availability_365', 'number_of_reviews',
            'review_scores_rating', 'review_scores_accuracy', 
            'review_scores_cleanliness', 'review_scores_checkin',
            'review_scores_communication', 'review_scores_location',
            'review_scores_value', 'host_listing_count']

# Converted columns to floating point
for feature_name in to_float:
    listings[feature_name] = listings[feature_name].astype(float)

In [11]:
# encode_categorical
# 
# Function to label encode categorical variables.
#     Input: array (array of values)
#     Output: array (array of encoded values)
def encode_categorical(array):
    if not array.dtype == np.dtype('float64'):
        return preprocessing.LabelEncoder().fit_transform(array) 
    else:
        return array

new_x = x.apply(encode_categorical)

NameError: ("global name 'preprocessing' is not defined", u'occurred at index id')

### Feature Emptiness and Erroneous Data
<p>Before imputing missing values, we should examine the percentage of values that are missing from each feature. Imputing data for a feature with too much missing data can bias the model.</p>

In [ ]:
# Returns percent of missing data in column
def percent_empty(df):
    
    bools = df.isnull().tolist()
    percent_empty = float(bools.count(True)) / float(len(bools))
    
    return percent_empty

# Store emptiness for all features
emptiness = []

# Get emptiness for all features
for i in range(0, listings.shape[1]):
    emptiness.append(round(percent_empty(listings.iloc[:,i]), 2))
    
empty_dict = dict(zip(listings.columns.values.tolist(), emptiness))

# Plot emptiness graph
empty = pd.DataFrame.from_dict(empty_dict, orient = 'index').sort_values(by=0)
ax = empty.plot(kind = 'bar', color='#E35A5C', figsize = (16, 5))
ax.set_xlabel('Predictor')
ax.set_ylabel('Percent Empty / NaN')
ax.set_title('Feature Emptiness')
ax.legend_.remove()

plt.show()

The percent emptiness graph shows that `square_feet` is over 90% empty. This is too empty for imputation, so we remove this feature. We also remove entries (rows) that have faulty data such as:

* There are 0 bedrooms
* There are 0 bathrooms
* There are 0 beds
* The price is \$0

In [ ]:
listings.drop('square_feet', axis=1, inplace=True)

# Delete bad entries
listings = listings[listings.bedrooms != 0]
listings = listings[listings.beds != 0]
listings = listings[listings.bathrooms != 0]
listings = listings[listings.price != 0]

print 'Number of entries removed: ', entries - listings.shape[0]
entries = listings.shape[0]

#### Trimming Neighborhood Entries

<p>When we explored our data we saw that geography was very important to pricing, especially on Manhattan. The `neighbourhood_cleansed` feature could therefore be important. Looking at the distribution below we notice it is heavily left-skewed.</p>

In [ ]:
# Get number of listings in neighborhoods
nb_counts = Counter(listings.neighbourhood_cleansed)
tdf = pd.DataFrame.from_dict(nb_counts, orient='index').sort_values(by=0)

# Plot number of listings in each neighborhood
ax = tdf.plot(kind='bar', figsize = (50,10), color = BNB_BLUE, alpha = 0.85)
ax.set_title("Neighborhoods by Number of Listings")
ax.set_xlabel("Neighborhood")
ax.set_ylabel("# of Listings")
plt.show()

print "Number of Neighborhoods:", len(nb_counts)

<p>We see that the majority of the neighborhoods have less than 100 listings. We currently have 186 neighborhoods - all of these categorical predictors when one-hot encoded will not be significant, so we will only keep neighborhoods with more than 100 listings. </p>

In [ ]:
# Delete neighborhoods with less than 100 entries
for i in nb_counts.keys():
    if nb_counts[i] < 100:
        del nb_counts[i]

tdf = pd.DataFrame.from_dict(nb_counts, orient='index').sort_values(by=0)
ax = tdf.plot(kind='bar', figsize = (22,4), color = BNB_BLUE, alpha = 0.85)
ax.set_title("Neighborhoods by House # (Top 48)")
ax.set_xlabel("Neighborhood")
ax.set_ylabel("# of Listings")

plt.show()

In [ ]:
listings.to_csv(path_or_buf='../clean.csv')

### Multicollinearity